### IMPORTACION DE LIBRERIAS


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.support import expected_conditions as EC
import socket
import time

### Configuracion del webdriver y navegacion a la pagina

In [ ]:
# Configura las opciones de Chrome
chrome_options = Options()
chrome_options.binary_location = "/usr/bin/google-chrome"  # Establece la ubicación del binario de Chrome

# Se configura el webdriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Abrir la página web
driver.get("https://www.everyonesinvited.uk/read")

### Configuracion celda del socket

In [ ]:
# Configurar el socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 12345)
sock.bind(server_address)
sock.listen(1)

# Aceptar una conexión del consumidor
connection, client_address = sock.accept()

### Recoleccion de mensajes y envio a traves del socket

In [ ]:
# Crear una espera explícita que se utilizará para esperar a que aparezca el botón
wait = WebDriverWait(driver, 10)

# Contador para el número de clics
clicks = 0

# Almacenar los mensajes que ya se han enviado
sent_messages = set()

# Crear una lista para almacenar los mensajes
messages_list = []

while clicks < 15:  # Ajustar este número según sea necesario
    try:
        # Esperar hasta que aparezca el botón y luego hacer clic en él
        load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="LOAD MORE"]')))
        load_more_button.click()

        # Incrementar el contador de clics
        clicks += 1

        # Esperar un poco para que se carguen los mensajes
        time.sleep(2)

        # Extraer y enviar los mensajes que se han cargado
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        messages = [p.text for p in soup.find_all('p')]
        for message in messages:
            if message not in sent_messages:
                connection.sendall((message + '\n').encode('utf-8'))
                sent_messages.add(message)
                messages_list.append(message)
                
        # Agregar declaración de impresión para verificar los mensajes extraídos
        print(messages)

    except Exception as e:
        # Si no se puede encontrar el botón, rompemos el bucle
        break


# Enviar una señal de cierre al consumidor
connection.sendall("SCRAPING_DONE".encode())

# Cerrar el navegador
driver.quit()

# Cerrar la conexión
connection.close()


### Escritura CSV

In [ ]:
#with open('mensajes.csv', 'w', newline='', encoding='utf-8') as csvfile:
#    writer = csv.writer(csvfile)
#    writer.writerow(['Mensaje'])
#    writer.writerows([[message] for message in messages_list])